In [1]:
from time import time
import tensorflow as tf
import cv2
import numpy as np
import os
from glob import glob
import math

# parser = argparse.ArgumentParser()
# parser.add_argument('--export', default=False, type=bool)
# parser.add_argument('--checkpoint', default='pix2pix/output/128x1024_v3')
# parser.add_argument('--strides', default=[64, 256], type=int, help='the strides for exported model')
# parser.add_argument('--input_dir', default='data/cell_data',
#                     help='directory to input images')
# parser.add_argument(
#     '--output_dir', default='output/run_method2', help='output image')
# args = parser.parse_args()

checkpoint = 'output/exported/'
output_dir = os.path.join(checkpoint, 'images')
input_dir = 'data/bprost/'
def get_tensor_by_name(name):
    name_on_device = '{}:0'.format(name)
    return tf.get_default_graph().get_tensor_by_name(name_on_device)

def load_image(path, verbal=False):
    print('---------------------------\nprocess:', path)
    name = path.split('/')[-1].split('.')[0]
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def write(path, image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return cv2.imwrite(path, image)

   

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:

tf.reset_default_graph()
from time import time
meta_path = os.path.join(checkpoint, 'export.meta')
print('meta path:', meta_path)

assert os.path.exists(meta_path), meta_path+' does not exist'

tf.train.import_meta_graph(meta_path)
sess = tf.Session()
saver = tf.train.Saver()

saver.restore(sess, tf.train.latest_checkpoint(
    checkpoint))

os.makedirs(output_dir, exist_ok=True)
start = time()
paths = glob('{}/*.JPG'.format(input_dir))
paths = [path for path in paths]
assert len(paths) > 0
print('Num of sample:', len(paths), input_dir)
inputs = get_tensor_by_name('inputs')
outputs = get_tensor_by_name('outputs')
start = time()
for path in paths:
    name = path.split('/')[-1].split('.')[0]
    image = load_image(path, verbal=True)
    output_image = sess.run(outputs, {inputs: image})
    merge_image = (0.5*image+0.5*output_image).astype('uint8')
    # os.makedirs('output/{}_{}'.format(args.stride, name), exist_ok=True)
    print('{}/{}_input.png'.format(output_dir, name))
    write('{}/{}_input.png'.format(output_dir, name), image)
    write('{}/{}_output.png'.format(output_dir, name), output_image)
    write('{}/{}_merge.png'.format(output_dir, name), merge_image)
    print(time()-start)
    start = time()
print('Running time:', time()-start)


meta path: output/exported/export.meta
INFO:tensorflow:Restoring parameters from output/exported/export
Num of sample: 9 data/bprost/
---------------------------
process: data/bprost/IMG_9263.JPG
output/exported/images/IMG_9263_input.png
3.788323163986206
---------------------------
process: data/bprost/IMG_9270.JPG
output/exported/images/IMG_9270_input.png
3.647564649581909
---------------------------
process: data/bprost/IMG_9269.JPG
output/exported/images/IMG_9269_input.png
3.621767997741699
---------------------------
process: data/bprost/IMG_9271.JPG
output/exported/images/IMG_9271_input.png
3.6516268253326416
---------------------------
process: data/bprost/IMG_9265.JPG
output/exported/images/IMG_9265_input.png
3.6664915084838867
---------------------------
process: data/bprost/IMG_9268.JPG
output/exported/images/IMG_9268_input.png
3.6316301822662354
---------------------------
process: data/bprost/IMG_9272.JPG
output/exported/images/IMG_9272_input.png
3.686495065689087
---------

In [4]:
np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])


13611523